# chapter 7 对象应用，可变性和垃圾回收



### 深复制

In [1]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [2]:
b

[[10, 20, [...]], 30]

In [3]:
from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]

### 函数的参数作为引用时

函数内部的形参是实参的别名

In [4]:
def f(a, b):
    a += b
    return a

In [5]:
x, y = 1, 2
f(x,y)

3

In [6]:
x,y

(1, 2)

In [7]:
x,y=[1,2,3],[4,5,6]
f(x,y)

[1, 2, 3, 4, 5, 6]

In [8]:
x,y # 内部的形参实际是该对象的一个引用

([1, 2, 3, 4, 5, 6], [4, 5, 6])

+ 不要使用可变类型作为参数的默认值

In [1]:
class HauntedBus:
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [2]:
bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)
bus2 = HauntedBus()
bus2.pick('Charlie')
print(bus2.passengers)
bus3 = HauntedBus()
print(bus3.passengers)
bus3.pick('Dave')
print(bus2.passengers)

bus2.passengers is bus3.passengers

bus1.passengers

['Alice', 'Bill']
['Bill', 'Charlie']
['Charlie']
['Charlie']
['Charlie', 'Dave']


['Bill', 'Charlie']

当没有指定初始乘客的实例时，会共享同一个乘客列表。默认值是可变对象，而且修改了它的值，那么后续函数调用都会受到影响。

一般使用None作为默认值，如果不是None，正确的实现方法是将passenger的副本复制给self.passenger

In [3]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

In [4]:
# 创建副本
class TwilightBus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

### del和垃圾回收



In [1]:
import weakref

s1 = {1, 2, 3}
s2 = s1
def bye():
    print("Gone with the wind……")

In [2]:
ender = weakref.finalize(s1, bye)
ender.alive

True

In [3]:
del s1
ender.alive

True

In [4]:
s2 = "spam"

Gone with the wind……


In [5]:
ender.alive

False

对元组t来说，t[:]不创建副本，而是返回同一个对象的引用。tuple(t)获得的也是同一个元组的引用。

In [6]:
t1 = (1,2,3)
t2 = tuple(t1)
t2 is t1

True

In [7]:
id(t1), id(t2)

(4522661496, 4522661496)

In [8]:
t2 = (3,4,5)

In [9]:
t1,t2

((1, 2, 3), (3, 4, 5))

In [10]:
t1 = (1,2,3)
t2 = (1,2,3)
t1  is t2

False

In [11]:
s1 = "ABC"
s2 = "ABC"
s1 is s2 # 字符串共享（驻留）

True